In [1]:
%matplotlib
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import imageio
import os

Using matplotlib backend: <object object at 0x000002B3CF1E64E0>


## Define core size and useful funcs

In [2]:
core_size = 18
print(core_size)

18


In [3]:
def rebin(arr, new_shape):
    shape = (new_shape[0], arr.shape[0] // new_shape[0],
             new_shape[1], arr.shape[1] // new_shape[1])
    return arr.reshape(shape).mean(-1).mean(1)

def plot_with_cores(core_locs,chosen_pic):
    fig2 = plt.figure(figsize=(24,13.5))
    ax2 = fig2.add_subplot(111)
    ax2.imshow(chosen_pic, cmap='Greys')

    for i in range(len(core_locs)):
        if i%2 == 0:
            col = 'black'
        else:
            col = 'black'
        patch_to_add = plt.Circle(core_locs[i], core_size, color=col, fill=False, linewidth=3.0,linestyle='--')
        ax2.add_patch(patch_to_add)
    #plt.axis('off')
    #ax2.set_axis_off()
    ax2.xaxis.set_major_locator(MultipleLocator(10))
    ax2.xaxis.set_minor_locator(MultipleLocator(5))
    ax2.yaxis.set_major_locator(MultipleLocator(10))
    ax2.yaxis.set_minor_locator(MultipleLocator(5))
    plt.grid(which='both')
    plt.show()
    return

# Can plot meshgrid to ensure each RGB component is not saturated
def RGB_plotter(chosen_pic):
    fig_prop = plt.figure(figsize=(12,10))
    ax_prop = plt.axes(projection='3d')

    x = np.arange(1080)
    y = np.arange(720)
    X, Y = np.meshgrid(x, y)
    ax_prop.plot_wireframe(X,Y,chosen_pic[:,:,0], color='blue')
    ax_prop.plot_wireframe(X,Y,chosen_pic[:,:,1],color='green')
    ax_prop.plot_wireframe(X,Y,chosen_pic[:,:,2],color='red')

    plt.show()
    return

# This function sums up the grayscale values in each core
# Giving an overall intensity value
def sum_intensity_in_cores(core_list,chosen_pic):
    
    copy_pic = np.copy(chosen_pic)
    x , y = np.ogrid[:copy_pic.shape[0] , :copy_pic.shape[1]] 

    core_sum_array = np.zeros(12)

    for core in range(12):
        dist_from_core_center = np.sqrt((x-core_list[core][1])**2 + (y-core_list[core][0])**2)
        circ_radius= core_size

        selection = dist_from_core_center < circ_radius
        core_sum_array[core] = sum(copy_pic[selection])
        copy_pic[selection] = 0.0
    return core_sum_array

# for image in pic_list_1:
#     RGB_plotter(image)
# RGB_plotter(bulk_pic)

## Label folder where images are

In [4]:
pic_list_1 = []

folder = 'Fibre Design/TPCF 4.2 160821/Characterisation/Invariant calculation/11_02_22_17cm/'
core_list = []
core_order
for filename in os.listdir(folder):
    print(filename)
    if filename[0] != '.':
        img = imageio.imread(os.path.join(folder, filename), pilmode ='F')
        pic_list_1.append(img)
        core_list.append((filename))
    
core_list = core_list[1:-1]
print(core_list)
binned_pic_list_1 = []

for pic in pic_list_1:
    bin_once = rebin(pic,(np.shape(pic)[0]//2, np.shape(pic)[1]//2))
    bin_twice = rebin(bin_once, (np.shape(bin_once)[0]//2, np.shape(bin_once)[1]//2))
    binned_pic_list_1.append(bin_twice)

binned_noise_pic = binned_pic_list_1[-1]
binned_list_without_noise_1 = []

for pic in binned_pic_list_1[1:-1]:
    binned_list_without_noise_1.append(pic-binned_noise_pic)
    


NameError: name 'core_order' is not defined

## Find core locations

In [5]:
def onclick(event):
    pos.append([event.xdata,event.ydata])

core_locs = []
i = 0

In [6]:
# Now we want to display the binned pic with some sort of grid to select the centers of each core
# Maybe this method will produce better pics

#for i in range(NUM_OF_PICS):
pos = []
fig1 = plt.figure(figsize=(24,13.5))
ax1 = fig1.add_subplot(111)

ax1.imshow(binned_pic_list_1[0], cmap='Reds')
ax1.xaxis.set_major_locator(MultipleLocator(10))
ax1.xaxis.set_minor_locator(MultipleLocator(5))
ax1.yaxis.set_major_locator(MultipleLocator(10))
ax1.yaxis.set_minor_locator(MultipleLocator(5))
#ax1.xaxis.set_major_formatter({x:.0f})
plt.grid(which='both')
#print(pic[308,505:530,2])
# plt.savefig('first paper/Figures/real_images/inverted blue Core {:d}.png'.format(index))
# plt.show()
fig1.canvas.mpl_connect('button_press_event', onclick)
fig1.show()

NameError: name 'binned_pic_list_1' is not defined

: 

In [8]:
# core_locs = np.zeros(NUM_OF_PICS)
if len([[int(i),int(j)] for i,j in pos]) == 12:
    core_locs.append([[int(i),int(j)] for i,j in pos])
#     core_locs[i] = [[int(i),int(j)] for i,j in pos]
    print([[int(i),int(j)] for i,j in pos])
i+=1
print(i)

[[290, 188], [276, 150], [298, 119], [337, 127], [352, 161], [366, 200], [343, 231], [317, 261], [279, 255], [240, 250], [226, 212], [211, 174]]
1


In [ ]:
# print(len(core_locs))
# for j,image in enumerate(binned_list_without_noise_1):
plot_with_cores(core_locs[0],binned_list_without_noise_1[0])


In [11]:
print(len(binned_list_without_noise_1))

0


In [9]:
core_intensity_list = []
# j = 0
for j,image in enumerate(binned_list_without_noise_1):
    intensity = sum_intensity_in_cores(core_locs[0],image)
    normalised_intensity = intensity/np.sum(intensity)
    print(normalised_intensity)
    normalised_intensity[normalised_intensity < 0] = 0
    core_intensity_list.append(normalised_intensity)



[ 1.45812879 -0.05252357 -0.03730906 -0.04471363 -0.01947896 -0.033154
 -0.02326814 -0.03952152 -0.03759091 -0.04403735 -0.06410387 -0.06242778]
[-0.00164488  0.08550204  0.03726409  0.09952957  0.05215711  0.07656875
  0.04839629  0.05568451  0.09968835  0.26996434  0.12269347  0.05419637]
[0.01040491 0.12609984 0.03441743 0.08179703 0.10306397 0.08765691
 0.17852387 0.0698966  0.08807328 0.07658843 0.14228877 0.00118894]
[0.00953899 0.01180495 0.01087315 0.02872241 0.01834125 0.10749324
 0.01940334 0.08450874 0.03645787 0.05079243 0.01057254 0.61149111]
[ 0.01303519  0.25183248  0.2041383   0.02771803  0.06945472  0.10004604
  0.00881784  0.07941406  0.11387531  0.04229705  0.09882717 -0.00945618]
[0.07841808 0.26905277 0.25943867 0.07501616 0.01842448 0.04654882
 0.00986162 0.05133713 0.06820925 0.04572415 0.07730562 0.00066324]
[ 0.00480144  0.08363186  0.17679681  0.23982488  0.15743476  0.03083918
  0.02158254 -0.00175337  0.00941719  0.09883404  0.17930725 -0.00071659]
[0.002644

In [16]:
def calc_i_d_list(unit_cell_index, core_intensity_lists):
    i_d_list = []
    unweighted_list = []
    for norm_vals in core_intensity_lists:
        a_norm = norm_vals[::2]
        b_norm = norm_vals[1::2]
        diff_list = a_norm - b_norm
        scaled_diff_list = diff_list*unit_cell_index

        i_d = np.sum(scaled_diff_list)
        unweighted_list.append(np.sum(diff_list))
        i_d_list.append(i_d)
        
    return i_d_list, unweighted_list

i_d_list, unweighted_id = calc_i_d_list(np.arange(1,7), core_intensity_list)

In [21]:
for i in range(len(core_list)):
    print(core_list[i][:-4]+' I_d: {:.2f}'.format(2*i_d_list[i]))

Core 1 I_d: 2.92
Core 10 I_d: -1.51
Core 11 I_d: 2.35
Core 12 I_d: -8.49
Core 2 I_d: 1.38
Core 3 I_d: 1.00
Core 4 I_d: 1.78
Core 5 I_d: -0.39
Core 6 I_d: -0.59
Core 7 I_d: -0.51
Core 8 I_d: -0.04
Core 9 I_d: 2.43


In [11]:
print(core_intensity_list)


[array([1.45812879, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ]), array([0.        , 0.08550204, 0.03726409, 0.09952957, 0.05215711,
       0.07656875, 0.04839629, 0.05568451, 0.09968835, 0.26996434,
       0.12269347, 0.05419637]), array([0.01040491, 0.12609984, 0.03441743, 0.08179703, 0.10306397,
       0.08765691, 0.17852387, 0.0698966 , 0.08807328, 0.07658843,
       0.14228877, 0.00118894]), array([0.00953899, 0.01180495, 0.01087315, 0.02872241, 0.01834125,
       0.10749324, 0.01940334, 0.08450874, 0.03645787, 0.05079243,
       0.01057254, 0.61149111]), array([0.01303519, 0.25183248, 0.2041383 , 0.02771803, 0.06945472,
       0.10004604, 0.00881784, 0.07941406, 0.11387531, 0.04229705,
       0.09882717, 0.        ]), array([0.07841808, 0.26905277, 0.25943867, 0.07501616, 0.01842448,
       0.04654882, 0.00986162, 0.05133713, 0.06820925, 0.04572415,
       0.07730562, 0.00066324]